# install all necessary libraries

In [ ]:
!pip install tensorboardX

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

# SET UP

In [1]:
import argparse
import os
import sys

sys.path.append('../..')
from pycorrector.seq2seq.data_reader import *
from pycorrector.seq2seq.train import *
from pycorrector.seq2seq.infer import *

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--raw_train_path",
                    default="../pycorrector/data/cn/sighan_2015/train.tsv", type=str,
                    help="The input data dir. Should contain the .tsv files (or other data files) for the task.",
                    )
parser.add_argument("--dataset", default="sighan", type=str,
                    help="Dataset name. selected in the list:" + ", ".join(["sighan", "cged"])
                    )
parser.add_argument("--use_segment", action="store_true", help="Whether not to segment train data")
parser.add_argument("--do_preprocess", action="store_true", default="True",help="Whether not to preprocess train data")
parser.add_argument("--segment_type", default="char", type=str,
                        help="Segment data type, selected in list: " + ", ".join(["char", "word"]))
parser.add_argument("--model_name_or_path",
                    default="bert-base-chinese", type=str,
                    help="Path to pretrained model or model identifier from huggingface.co/models",
                    )
parser.add_argument("--model_dir", default="output/RNA/", type=str, help="Dir for model save.")
parser.add_argument("--arch", default="seq2seq", type=str,
                    help="The name of the task to train selected in the list: " + ", ".join(
                        ['seq2seq', 'convseq2seq', 'bertseq2seq']),
                    )
parser.add_argument("--train_path", default="output/train.txt", type=str, help="Train file after preprocess.")
parser.add_argument("--test_path", default="output/test.txt", type=str, help="Test file after preprocess.")
parser.add_argument("--max_length", default=128, type=int,
                    help="The maximum total input sequence length after tokenization. \n"
                            "Sequences longer than this will be truncated, sequences shorter padded.",
                    )
parser.add_argument("--batch_size", default=32, type=int, help="Batch size.")
parser.add_argument("--embed_size", default=128, type=int, help="Embedding size.")
parser.add_argument("--hidden_size", default=128, type=int, help="Hidden size.")
parser.add_argument("--dropout", default=0.25, type=float, help="Dropout rate.")
parser.add_argument("--epochs", default=10, type=int, help="Epoch num.")

args = parser.parse_args([])

In [5]:
data = get_data_file(args.raw_train_path, args.use_segment, args.segment_type)[:1000]
data1 = get_data_file("../pycorrector/data/RNA/train", args.use_segment, args.segment_type)[:1000]
for i in range(1000):
    [a,b] = data1[i]
    if a!=b:
        print(i)
        break

        

15


In [6]:
data1[15]

['GLY GLU ALA SER PRO VAL ASP PRO LEU ARG PRO VAL VAL ASP ALA SER ILE GLN PRO LEU LEU LYS GLU HIS ARG ILE PRO GLY MET ALA VAL ALA VAL LEU LYS ASP GLY LYS ALA HIS TYR PHE ASN TYR GLY VAL ALA ASN ARG GLU SER GLY ALA GLY VAL SER GLU GLN THR LEU PHE GLU ILE GLY SER VAL SER LYS THR LEU THR ALA THR LEU GLY ALA TYR ALA VAL VAL LYS GLY ALA MET GLN LEU ASP ASP LYS ALA SER ARG HIS ALA PRO TRP LEU LYS GLY SER ALA PHE ASP SER ILE THR MET GLY GLU LEU ALA THR TYR SER ALA GLY GLY LEU PRO LEU GLN PHE PRO GLU GLU VAL ASP SER SER GLU LYS MET ARG ALA TYR TYR ARG GLN TRP ALA PRO VAL TYR SER PRO GLY SER HIS ARG GLN TYR SER ASN PRO SER ILE GLY LEU PHE GLY HIS LEU ALA ALA SER SER LEU LYS GLN PRO PHE ALA PRO LEU MET GLU GLN THR LEU LEU PRO GLY LEU GLY MET HIS HIS THR TYR VAL ASN VAL PRO LYS GLN ALA MET ALA SER TYR ALA TYR GLY TYR SER LYS GLU ASP LYS PRO ILE ARG VAL ASN PRO GLY MET LEU ALA ASP GLU ALA TYR GLY ILE LYS THR SER SER ALA ASP LEU LEU ARG PHE VAL LYS ALA ASN ILE GLY GLY VAL ASP ASP LYS ALA LEU GLN GL

# data_reader.py

In [7]:
# Define constants associated with the usual special tokens.
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'
UNK_TOKEN = '<unk>'
PAD_TOKEN = '<pad>'
class CscDataset(object):
    def __init__(self, file_path):
        self.data = json.load(open(file_path, 'r', encoding='utf-8'))

    def load(self):
        data_list = []
        for item in self.data:
            data_list.append(item['original_text'] + '\t' + item['correct_text'])
        return data_list

def create_dataset(path, num_examples=None, split_on_space=False):
    if path.endswith('.json'):
        d = CscDataset(path)
        lines = d.load()
    else:
        lines = open(path, 'r', encoding='utf-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(s, split_on_space) for s in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)


def preprocess_sentence(sentence, split_on_space=False):
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    return [SOS_TOKEN] + sentence.split() if split_on_space else list(sentence) + [EOS_TOKEN]


In [10]:
from collections import Counter
'''
class Counter(object):
    def __init__(self):
        self.counter = {}
    def update(self, token):
'''    

def read_vocab(input_texts, max_size=None, min_count=0):
    token_counts = Counter()
    special_tokens = [PAD_TOKEN, UNK_TOKEN, SOS_TOKEN, EOS_TOKEN]
    '''
    for texts in input_texts:
        for token in texts:
            token_counts.update(token)
    '''
    for texts in input_texts:
        for token in texts:
            token_counts.update(token)
    # Sort word count by value
    count_pairs = token_counts.most_common()
    vocab = [k for k, v in count_pairs if v >= min_count]
    print(vocab)
    # Insert the special tokens to the beginning
    vocab[0:0] = special_tokens
    full_token_id = list(zip(vocab, range(len(vocab))))[:max_size]
    vocab2id = dict(full_token_id)
    return vocab2id

In [11]:
def one_hot(src_sentences, trg_sentences, src_dict, trg_dict, sort_by_len=True):
    """vector the sequences.
    """

    out_src_sentences = [[src_dict.get(w, 0) for w in sent] for sent in src_sentences]
    out_trg_sentences = [[trg_dict.get(w, 0) for w in sent] for sent in trg_sentences]

    # sort sentences by english lengths
    def len_argsort(seq):
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))

    # sort length
    if sort_by_len:
        sorted_index = len_argsort(out_src_sentences)
        out_src_sentences = [out_src_sentences[i] for i in sorted_index]
        out_trg_sentences = [out_trg_sentences[i] for i in sorted_index]

    return out_src_sentences, out_trg_sentences

In [12]:
# -*- coding: utf-8 -*-
"""
@author:XuMing(xuming624@qq.com)
@description: 
"""

import torch
import torch.nn as nn
import torch.nn.functional as F


class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Encoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        '''
            torch.nn.Embedding(numembeddings, embeddingdim)
                * numembeddings代表一共有多少个词
                * embedding_dim代表每个词创建一个多少维的向量来表示他
        '''
        self.rnn = nn.GRU(embed_size, enc_hidden_size, batch_first=True, bidirectional=True)
        '''
            torch.nn.GRU(input_size, hidden_size, num_layers, bias,batch_first,dropout,bidirectional)
                * input_size: the number of expected features in the input x
                * hidden_size: the number of features in the hidden state h
                * batch_first: if True, then (batch, seq, feature), else (seq, batch, feature)
                * bidirectional: if True, becomes a bidirectional GRU. Default: False
            
        '''
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(enc_hidden_size * 2, dec_hidden_size)

    def forward(self, x, lengths):
        # 将x根据长度来排序
        sorted_len, sorted_idx = lengths.sort(0, descending=True)
        x_sorted = x[sorted_idx.long()]
        embedded = self.dropout(self.embed(x_sorted))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(),
                                                            batch_first=True)
        '''
            https://zhuanlan.zhihu.com/p/34418001
            torch.nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=False,enforce_sorted=True)
            当我们进行batch个训练数据一起计算的时候，我们会遇到多个训练样例长度不同时的情况，这样我们就会很自然的进行padding，
            将短句子padding为跟最长的句子一样
            
            pytorch中RNN处理变长padding主要用torch.nn.utils.rnn.pack_padded_sequence()以及torch.nn.utils.rnn.pad_packed_sequence()来进行。
            
            
            
        '''
        packed_out, hid = self.rnn(packed_embedded)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        _, original_idx = sorted_idx.sort(0, descending=False)
        out = out[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()

        hid = torch.cat([hid[-2], hid[-1]], dim=1)
        hid = torch.tanh(self.fc(hid)).unsqueeze(0)

        return out, hid


class Attention(nn.Module):
    """
    Luong Attention,根据context vectors和当前的输出hidden states，计算输出
    """

    def __init__(self, enc_hidden_size, dec_hidden_size):
        super(Attention, self).__init__()

        self.enc_hidden_size = enc_hidden_size
        self.dec_hidden_size = dec_hidden_size

        self.linear_in = nn.Linear(enc_hidden_size * 2, dec_hidden_size, bias=False)
        self.linear_out = nn.Linear(enc_hidden_size * 2 + dec_hidden_size, dec_hidden_size)

    def forward(self, output, context, mask):
        # output: batch_size, output_len, dec_hidden_size
        # context: batch_size, context_len, 2*enc_hidden_size

        batch_size = output.size(0)
        output_len = output.size(1)
        input_len = context.size(1)

        context_in = self.linear_in(context.view(batch_size * input_len, -1)).view(
            batch_size, input_len, -1)  # batch_size, context_len, dec_hidden_size

        # context_in.transpose(1,2): batch_size, dec_hidden_size, context_len
        # output: batch_size, output_len, dec_hidden_size
        attn = torch.bmm(output, context_in.transpose(1, 2))
        # batch_size, output_len, context_len

        attn.data.masked_fill(mask, -1e6)

        attn = F.softmax(attn, dim=2)
        # batch_size, output_len, context_len

        context = torch.bmm(attn, context)
        # batch_size, output_len, enc_hidden_size

        output = torch.cat((context, output), dim=2)  # batch_size, output_len, hidden_size*2

        output = output.view(batch_size * output_len, -1)
        output = torch.tanh(self.linear_out(output))
        output = output.view(batch_size, output_len, -1)
        return output, attn


class Decoder(nn.Module):
    """
    decoder会根据已经翻译的句子内容，和context vectors，来决定下一个输出的单词
    """

    def __init__(self, vocab_size, embed_size, enc_hidden_size, dec_hidden_size, dropout=0.2):
        super(Decoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.attention = Attention(enc_hidden_size, dec_hidden_size)
        self.rnn = nn.GRU(embed_size, enc_hidden_size, batch_first=True)
        self.out = nn.Linear(dec_hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def create_mask(self, x_len, y_len):
        # a mask of shape x_len * y_len
        max_x_len = x_len.max()
        max_y_len = y_len.max()
        x_mask = torch.arange(max_x_len, device=x_len.device)[None, :] < x_len[:, None]
        y_mask = torch.arange(max_y_len, device=x_len.device)[None, :] < y_len[:, None]
        mask = ~ x_mask[:, :, None] * y_mask[:, None, :]
        return mask

    def forward(self, ctx, ctx_lengths, y, y_lengths, hid):
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)
        y_sorted = y[sorted_idx.long()]
        hid = hid[:, sorted_idx.long()]

        y_sorted = self.dropout(self.embed(y_sorted))  # batch_size, output_length, embed_size

        packed_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(packed_seq, hid)
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        _, original_idx = sorted_idx.sort(0, descending=False)
        output_seq = unpacked[original_idx.long()].contiguous()
        hid = hid[:, original_idx.long()].contiguous()

        mask = self.create_mask(y_lengths, ctx_lengths)

        output, attn = self.attention(output_seq, ctx, mask)
        output = F.log_softmax(self.out(output), -1)

        return output, hid, attn


class Seq2Seq(nn.Module):
    """
    Seq2Seq, 最后我们构建Seq2Seq模型把encoder, attention, decoder串到一起
    """

    def __init__(self,
                 encoder_vocab_size,
                 decoder_vocab_size,
                 embed_size,
                 enc_hidden_size,
                 dec_hidden_size,
                 dropout,
                 ):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(vocab_size=encoder_vocab_size,
                               embed_size=embed_size,
                               enc_hidden_size=enc_hidden_size,
                               dec_hidden_size=dec_hidden_size,
                               dropout=dropout)
        self.decoder = Decoder(vocab_size=decoder_vocab_size,  # len(trg_2_ids),
                               embed_size=embed_size,
                               enc_hidden_size=enc_hidden_size,
                               dec_hidden_size=dec_hidden_size,
                               dropout=dropout)

    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)
        output, hid, attn = self.decoder(ctx=encoder_out,
                                         ctx_lengths=x_lengths,
                                         y=y,
                                         y_lengths=y_lengths,
                                         hid=hid)
        return output, attn

    def translate(self, x, x_lengths, y, max_length=128):
        print(len(x))
        encoder_out, hid = self.encoder(x, x_lengths)
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):
            output, hid, attn = self.decoder(ctx=encoder_out,
                                             ctx_lengths=x_lengths,
                                             y=y,
                                             y_lengths=torch.ones(batch_size).long().to(y.device),
                                             hid=hid)
            
            y = output.max(2)[1].view(batch_size, 1)
            preds.append(y)
            
            attns.append(attn)
        return torch.cat(preds, 1), torch.cat(attns, 1)


class LanguageModelCriterion(nn.Module):
    """
    masked cross entropy loss
    """

    def __init__(self):
        super(LanguageModelCriterion, self).__init__()

    def forward(self, input, target, mask):
        # input: (batch_size * seq_len) * vocab_size
        input = input.contiguous().view(-1, input.size(2))
        # target: batch_size * 1
        target = target.contiguous().view(-1, 1)
        mask = mask.contiguous().view(-1, 1)
        output = -input.gather(1, target) * mask
        output = torch.sum(output) / torch.sum(mask)

        return output


In [13]:
unk_tokens = [' ', '“', '”', '‘', '’', '琊', '\n', '…', '—', '擤', '\t', '֍', '玕', '', '《', '》']


def get_errors(corrected_text, origin_text):
    sub_details = []
    for i, ori_char in enumerate(origin_text):
        if i >= len(corrected_text):
            continue
        if ori_char in unk_tokens:
            # deal with unk word
            corrected_text = corrected_text[:i] + ori_char + corrected_text[i:]
            continue
        if ori_char != corrected_text[i]:
            sub_details.append((ori_char, corrected_text[i], i, i + 1))
    sub_details = sorted(sub_details, key=operator.itemgetter(2))
    return corrected_text, sub_details


class Inference(object):
    def __init__(self, model_dir, arch='convseq2seq',
                 embed_size=128, hidden_size=128, dropout=0.25, max_length=128):
        logger.debug("Device: {}".format(device))
        logger.debug(f'Use {arch} model.')
        if arch in ['seq2seq', 'convseq2seq']:
            src_vocab_path = os.path.join(model_dir, 'vocab_source.txt')
            trg_vocab_path = os.path.join(model_dir, 'vocab_target.txt')
            self.src_2_ids = load_word_dict(src_vocab_path)
            self.trg_2_ids = load_word_dict(trg_vocab_path)
            self.id_2_trgs = {v: k for k, v in self.trg_2_ids.items()}
            if arch == 'seq2seq':
                self.model = Seq2Seq(encoder_vocab_size=len(self.src_2_ids),
                                     decoder_vocab_size=len(self.trg_2_ids),
                                     embed_size=embed_size,
                                     enc_hidden_size=hidden_size,
                                     dec_hidden_size=hidden_size,
                                     dropout=dropout).to(device)
                model_path = os.path.join(model_dir, 'seq2seq.pth')
                logger.debug('Load model from {}'.format(model_path))
                self.model.load_state_dict(torch.load(model_path, map_location=device))
                self.model.eval()
            else:
                trg_pad_idx = self.trg_2_ids[PAD_TOKEN]
                self.model = ConvSeq2Seq(encoder_vocab_size=len(self.src_2_ids),
                                         decoder_vocab_size=len(self.trg_2_ids),
                                         embed_size=embed_size,
                                         enc_hidden_size=hidden_size,
                                         dec_hidden_size=hidden_size,
                                         dropout=dropout,
                                         trg_pad_idx=trg_pad_idx,
                                         device=device,
                                         max_length=max_length).to(device)
                model_path = os.path.join(model_dir, 'convseq2seq.pth')
                self.model.load_state_dict(torch.load(model_path, map_location=device))
                logger.debug('Load model from {}'.format(model_path))
                self.model.eval()
        elif arch == 'bertseq2seq':
            # Bert Seq2seq model
            use_cuda = True if torch.cuda.is_available() else False

            # encoder_type=None, encoder_name=None, decoder_name=None
            self.model = Seq2SeqModel("bert", "{}/encoder".format(model_dir),
                                      "{}/decoder".format(model_dir), use_cuda=use_cuda)
        else:
            logger.error('error arch: {}'.format(arch))
            raise ValueError("Model arch choose error. Must use one of seq2seq model.")
        self.arch = arch
        self.max_length = max_length

    def predict(self, sentence_list):
        result = []
        if self.arch in ['seq2seq', 'convseq2seq']:
            for query in sentence_list:
                out = []
                tokens = query.split()
                tokens = [SOS_TOKEN] + tokens + [EOS_TOKEN]
                src_ids = [self.src_2_ids[i] for i in tokens if i in self.src_2_ids]

                sos_idx = self.trg_2_ids[SOS_TOKEN]
                if self.arch == 'seq2seq':
                    src_tensor = torch.from_numpy(np.array(src_ids).reshape(1, -1)).long().to(device)
                    src_tensor_len = torch.from_numpy(np.array([len(src_ids)])).long().to(device)
                    sos_tensor = torch.Tensor([[self.trg_2_ids[SOS_TOKEN]]]).long().to(device)
                    translation, attn = self.model.translate(src_tensor, src_tensor_len, sos_tensor, self.max_length)
                    translation = [self.id_2_trgs[i] for i in translation.data.cpu().numpy().reshape(-1) if
                                   i in self.id_2_trgs]
                else:
                    src_tensor = torch.from_numpy(np.array(src_ids).reshape(1, -1)).long().to(device)
                    translation, attn = self.model.translate(src_tensor, sos_idx)
                    translation = [self.id_2_trgs[i] for i in translation if i in self.id_2_trgs]
                for word in translation:
                    if word != EOS_TOKEN:
                        out.append(word)
                    else:
                        break
                corrected_text = ''.join(out)
                corrected_text, sub_details = get_errors(corrected_text, query)
                result.append([corrected_text, sub_details])
        else:
            corrected_sents = self.model.predict(sentence_list)
            corrected_sents = [i.replace(' ', '') for i in corrected_sents]
            for c, s in zip(corrected_sents, sentence_list):
                c = c.replace(' ', '')
                c, sub_details = get_errors(c, s)
                result.append([c, sub_details])
        return result


In [14]:
def train(arch, train_path, batch_size, embed_size, hidden_size, dropout, epochs,
          model_dir, max_length, use_segment, model_name_or_path):
    logger.info("device: {}".format(device))
    arch = arch.lower()
    logger.debug(f'use {arch} model.')
    os.makedirs(model_dir, exist_ok=True)
    if arch in ['seq2seq', 'convseq2seq']:
        src_vocab_path = os.path.join(model_dir, 'vocab_source.txt')
        trg_vocab_path = os.path.join(model_dir, 'vocab_target.txt')

        source_texts, target_texts = create_dataset(train_path, split_on_space=True)
        logger.debug("source_texts:",source_texts)
        #src_2_ids = read_vocab(source_texts)
        #trg_2_ids = read_vocab(target_texts)
        #print(src_2_ids)
        #save_word_dict(src_2_ids, src_vocab_path)
        #save_word_dict(trg_2_ids, trg_vocab_path)

        src_2_ids = load_word_dict(src_vocab_path)
        trg_2_ids = load_word_dict(trg_vocab_path)
        
        id_2_srcs = {v: k for k, v in src_2_ids.items()}
        id_2_trgs = {v: k for k, v in trg_2_ids.items()}
        train_src, train_trg = one_hot(source_texts, target_texts, src_2_ids, trg_2_ids, sort_by_len=True)

        logger.debug(f'src: {[id_2_srcs[i] for i in train_src[0]]}')
        logger.debug(f'trg: {[id_2_trgs[i] for i in train_trg[0]]}')

        train_data = gen_examples(train_src, train_trg, batch_size, max_length)

        if arch == 'seq2seq':
            # Normal seq2seq
            model = Seq2Seq(encoder_vocab_size=len(src_2_ids),
                            decoder_vocab_size=len(trg_2_ids),
                            embed_size=embed_size,
                            enc_hidden_size=hidden_size,
                            dec_hidden_size=hidden_size,
                            dropout=dropout).to(device)
            logger.info(model)
            loss_fn = LanguageModelCriterion().to(device)
            optimizer = torch.optim.Adam(model.parameters())

            train_seq2seq_model(model, train_data, device, loss_fn, optimizer, model_dir, epochs=epochs)
        else:
            # Conv seq2seq model
            trg_pad_idx = trg_2_ids[PAD_TOKEN]
            model = ConvSeq2Seq(encoder_vocab_size=len(src_2_ids),
                                decoder_vocab_size=len(trg_2_ids),
                                embed_size=embed_size,
                                enc_hidden_size=hidden_size,
                                dec_hidden_size=hidden_size,
                                dropout=dropout,
                                trg_pad_idx=trg_pad_idx,
                                device=device,
                                max_length=max_length).to(device)
            logger.info(model)
            loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_idx)
            optimizer = torch.optim.Adam(model.parameters())

            train_convseq2seq_model(model, train_data, device, loss_fn, optimizer, model_dir, epochs=epochs)
    elif arch == 'bertseq2seq':
        # Bert Seq2seq model
        model_args = {
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "max_seq_length": max_length if max_length else 128,
            "train_batch_size": batch_size if batch_size else 8,
            "num_train_epochs": epochs if epochs else 10,
            "save_eval_checkpoints": False,
            "save_model_every_epoch": False,
            "silent": False,
            "evaluate_generated_text": True,
            "evaluate_during_training": True,
            "evaluate_during_training_verbose": True,
            "best_model_dir": os.path.join(model_dir, 'best_model'),
            "use_multiprocessing": False,
            "save_best_model": True,
            "max_length": max_length if max_length else 128,  # The maximum length of the sequence to be generated.
            "output_dir": model_dir if model_dir else "./output/bertseq2seq/",
        }

        use_cuda = True if torch.cuda.is_available() else False
        # encoder_type=None, encoder_name=None, decoder_name=None
        # encoder_name="bert-base-chinese"
        model = Seq2SeqModel("bert", model_name_or_path, model_name_or_path, args=model_args, use_cuda=use_cuda)

        logger.info('start train bertseq2seq ...')
        data = load_bert_data(train_path, use_segment)
        logger.info(f'load data done, data size: {len(data)}')
        logger.debug(f'data samples: {data[:10]}')
        train_data, dev_data = train_test_split(data, test_size=0.1, shuffle=False)

        train_df = pd.DataFrame(train_data, columns=['input_text', 'target_text'])
        dev_df = pd.DataFrame(dev_data, columns=['input_text', 'target_text'])

        def count_matches(labels, preds):
            logger.debug(f"labels: {labels[:10]}")
            logger.debug(f"preds: {preds[:10]}")
            match = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
            logger.debug(f"match: {match}")
            return match

        model.train_model(train_df, eval_data=dev_df, matches=count_matches)
    else:
        logger.error('error arch: {}'.format(arch))
        raise ValueError("Model arch choose error. Must use one of seq2seq model.")

In [20]:
if args.do_preprocess:
    # Preprocess
    data_list = []
    os.makedirs(args.model_dir, exist_ok=True)
    '''
    if args.dataset == 'sighan':
        data_list.extend(get_data_file(args.raw_train_path, args.use_segment, args.segment_type))
    else:
        data_list.extend(parse_xml_file(args.raw_train_path, args.use_segment, args.segment_type))
    '''
    data_list.extend(get_data_file("../pycorrector/data/RNA/train", args.use_segment, args.segment_type))
    if data_list:
        save_corpus_data(data_list, args.train_path, args.test_path)
# Train model with train data file
train(args.arch,
        args.train_path,
        args.batch_size,
        args.embed_size,
        args.hidden_size,
        args.dropout,
        args.epochs,
        args.model_dir,
        args.max_length,
        args.use_segment,
        args.model_name_or_path,
        )

KeyboardInterrupt: 

In [19]:

m = Inference(args.model_dir,
                  args.arch,
                  embed_size=args.embed_size,
                  hidden_size=args.hidden_size,
                  dropout=args.dropout,
                  max_length=args.max_length
                  )
inputs = [
    'MET LYS LYS LEU GLN ILE ALA VAL GLY ILE ILE ARG ASN GLU ASN ASN GLU ILE PHE ILE THR ARG ARG ALA ALA ASP ALA HIS MET ALA ASN LYS LEU GLU PHE PRO GLY GLY LYS ILE GLU MET GLY GLU THR PRO GLU GLN ALA VAL VAL ARG GLU LEU GLN GLU GLU VAL GLY ILE THR PRO GLN HIS PHE SER LEU PHE GLU LYS LEU GLU TYR GLU PHE PRO ASP ARG HIS ILE THR LEU TRP PHE TRP LEU VAL GLU ARG TRP GLU GLY GLU PRO TRP GLY LYS GLU GLY GLN PRO GLY GLU TRP MET SER LEU VAL GLY LEU ASN ALA ASP ASP PHE PRO PRO ALA ASN GLU PRO VAL ILE ALA LYS LEU LYS ARG LEU',
    'ASP ALA ILE ALA ASP ALA SER LYS ARG PHE SER ASP ALA THR TYR PRO ILE ALA GLU LYS PHE ASP TRP GLY GLY SER SER ALA ILE ALA LYS TYR ILE ALA ASP ALA SER ALA GLY ASN PRO ARG GLN ALA ALA LEU ALA VAL GLU LYS LEU LEU GLU VAL GLY LEU THR MET ASP PRO LYS LEU VAL ARG ALA ALA VAL GLU ALA HIS SER LYS ALA LEU ASP SER ALA LYS LYS ASN ALA LYS LEU MET ALA SER LYS GLU ASP PHE ALA ALA VAL ASN GLU ALA LEU ALA ARG MET ILE ALA SER ALA ASP LYS GLN LYS PHE ALA ALA LEU ARG THR ALA PHE PRO GLU SER ARG GLU LEU GLN GLY LYS LEU PHE ALA GLY ASN ASN ALA PHE GLU ALA GLU LYS ALA TYR ASP SER PHE LYS ALA LEU THR SER ALA VAL ARG ASP ALA SER ILE ASN GLY ALA LYS ALA PRO VAL ILE ALA GLU ALA ALA ARG ALA GLU ARG TYR VAL GLY ASP GLY PRO VAL GLY ARG ALA ALA LYS LYS PHE SER GLU ALA THR TYR PRO ILE MET ASP LYS LEU ASP TRP GLY LYS SER PRO GLU ILE SER LYS TYR ILE GLU THR ALA SER ALA LYS ASN PRO LYS MET MET ALA ASP GLY ILE ASP LYS THR LEU GLU VAL ALA LEU THR MET ASN GLN ASN ALA ILE ASN ASP ALA VAL PHE ALA HIS VAL ARG ALA ILE LYS GLY ALA LEU ASN THR PRO GLY LEU VAL ALA GLU ARG ASP ASP PHE ALA ARG VAL ASN LEU ALA LEU ALA LYS MET ILE ALA THR ALA ASP PRO ALA LYS PHE LYS ALA LEU LEU THR ALA PHE PRO GLY ASN ALA ASP LEU GLN MET ALA LEU PHE ALA ALA ASN ASN PRO GLU GLN ALA LYS ALA ALA TYR GLU THR PHE VAL ALA LEU THR SER ALA VAL ALA SER SER THR'
]
outputs = m.predict(inputs)

for a, b in zip(inputs, outputs):
    print('input  :', a)
    print('predict:', b[0], b[1])
    print()



2022-09-12 22:42:00.156 | DEBUG    | __main__:__init__:22 - Device: cpu
2022-09-12 22:42:00.157 | DEBUG    | __main__:__init__:23 - Use seq2seq model.
2022-09-12 22:42:00.163 | DEBUG    | __main__:__init__:38 - Load model from output/RNA/seq2seq.pth


1
1
input  : MET LYS LYS LEU GLN ILE ALA VAL GLY ILE ILE ARG ASN GLU ASN ASN GLU ILE PHE ILE THR ARG ARG ALA ALA ASP ALA HIS MET ALA ASN LYS LEU GLU PHE PRO GLY GLY LYS ILE GLU MET GLY GLU THR PRO GLU GLN ALA VAL VAL ARG GLU LEU GLN GLU GLU VAL GLY ILE THR PRO GLN HIS PHE SER LEU PHE GLU LYS LEU GLU TYR GLU PHE PRO ASP ARG HIS ILE THR LEU TRP PHE TRP LEU VAL GLU ARG TRP GLU GLY GLU PRO TRP GLY LYS GLU GLY GLN PRO GLY GLU TRP MET SER LEU VAL GLY LEU ASN ALA ASP ASP PHE PRO PRO ALA ASN GLU PRO VAL ILE ALA LYS LEU LYS ARG LEU
predict: MET LYS LYS LEU GLN ILE ALA VAL GLY ILE ILE ARG ASN GLU ASN ASN GLU ILE PHE ILE THR ARG ARG ALA ALA ASP ALA HIS MET ALA ASN LYS LEU GLU PHE PRO GLY GLY LYS ILE GLU MET GLY GLU THR PRO GLU GLN ALA VAL VAL ARG GLU LEU GLN GLU GLU VAL GLY ILE THR PRO GLN HIS PHE SER LEU PHE GLU LYS LEU GLU TYR GLU PHE PRO ASP ARG HIS ILE THR LEU TRP PHE TRP LEU VAL GLU ARG TRP GLU GLY GLU PRO TRP GLY LYS GLU GLY GLN PRO GLY GLU TRP MET SER LEU VAL GLY LEU ASN ALA ASP ASP PHE PR